In [ ]:
!pip install wfdb

In [ ]:
import wfdb
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# Get All record  waveforms list

from datetime import datetime

f = open("xxx/RECORDS-waveforms")
line = f.readline()
data=[]
while line:
    data.append(line.split('\n')[0])
    line = f.readline()
f.close()

data_dic={}
for i in data:
    key=i.split('/')[1]
    if key in data_dic.keys():
        data_dic[key].append('-'.join(i.split('/')[-1].split('n')[0].split('-')[1:]))
    else:
        data_dic[key]=[]
        data_dic[key].append('-'.join(i.split('/')[-1].split('n')[0].split('-')[1:]))


len_size = []
key_list = []
all_data=[]
for key in data_dic.keys():
    if len(data_dic[key]) > 1:
        split_data =data_dic[key][0].split('-')
        year = split_data[0]
        mon = split_data[1]
        day = split_data[2]
        hour = split_data[3]
        minu = split_data[4]
        date_0 = datetime.strptime(year+'-' +mon+'-' +day+' ' + hour+':' +minu+':0', '%Y-%m-%d %H:%M:%S')
        
        
        key_list.append(key)
        split_data =data_dic[key][-1].split('-')
        year = split_data[0]
        mon = split_data[1]
        day = split_data[2]
        hour = split_data[3]
        minu = split_data[4]
        date_1 = datetime.strptime(year+'-' +mon+'-' +day+' ' + hour+':' +minu+':0', '%Y-%m-%d %H:%M:%S')   
        
        len_size.append(date_1-date_0)
        all_data.append((date_1-date_0, key))
    else:
        continue
        
all_data.sort()

In [2]:
import pickle
file_open=open('xxx/Extracted_signal_records.pl','rb')
all_data_dic=pickle.load(file_open)
file_open.close()


In [3]:
all_data_dic.keys()

dict_keys(['p007809', 'p018846', 'p013628', 'p006621', 'p017026', 'p019975', 'p018996', 'p024455', 'p008799', 'p025178', 'p026747', 'p008228', 'p053036', 'p023707', 'p062186', 'p007241', 'p025581', 'p027551', 'p006428', 'p025329', 'p067301', 'p053102', 'p079414', 'p004113', 'p011861', 'p090398', 'p062681', 'p024810', 'p040972', 'p000402', 'p000689', 'p080260', 'p076558', 'p066880', 'p020486', 'p045492', 'p052453', 'p077220', 'p023657', 'p064160', 'p008949', 'p049649', 'p029866', 'p074640', 'p019059', 'p024099', 'p099085', 'p024084', 'p059991', 'p056287', 'p041702', 'p030637', 'p073713', 'p071862', 'p025635', 'p060767', 'p079923', 'p078023', 'p056502', 'p048388', 'p044206', 'p042763', 'p082178', 'p049480', 'p086824', 'p013830', 'p092063', 'p047956', 'p053835', 'p046510', 'p030669', 'p093078', 'p088726', 'p058163', 'p096260', 'p013329', 'p061898', 'p094503', 'p029035', 'p031029', 'p055357', 'p069857', 'p042820', 'p080180', 'p059889', 'p053247', 'p047747', 'p055094', 'p042510', 'p070886',

In [4]:
len(all_data_dic.keys())

247

In [5]:
all_data_dic['p007809'].keys()

dict_keys(['2136-08-31-17-33', '2137-05-28-16-04', '2137-12-21-19-03', '2138-11-10-11-14', '2139-03-04-21-25', '2139-07-02-18-51', '2141-05-15-02-42'])

In [6]:
all_data_dic['p007809']['2136-08-31-17-33']

['29:40:10-30:11:50', '30:18:40-30:24:20', '51:26:40-51:29:00']

In [ ]:
subj_dic = all_data_dic.copy()

In [7]:
# import pickle
# data=[]
# file_open=open('Extracted_signal_records.pl','wb')
# pickle.dump(all_data_dic,file_open)
# file_open.close()

In [ ]:
# download data

for arr_data in all_data_dic.keys():

  db_dir = 'mimic3wdb-matched' 
  dl_dir = 'xxx/MIMICIII/'

  file_name = arr_data
  record_name = file_name
  file_name = file_name[:3]+'/'+file_name+'/'+record_name+'-'
  file_name

  record_list = []
  for i in data_dic[record_name]:
      record_list.append(file_name+i)

  records = record_list
  wfdb.dl_database(db_dir,dl_dir,records)

In [ ]:
import os
g = os.walk(r'xxx/MIMICIII/')  
folder_list = []
for path,dir_list,file_list in g: 
    for dir_name in dir_list:
        folder_list.append(os.path.join(path, dir_name).split('/')[-1])

In [ ]:
import wfdb
import datetime
import wfdb
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
from tqdm import tqdm

data_dic={}
for subjkey in tqdm(all_data_dic.keys()) :
    print(subjkey)
    if  subjkey in already_arr or subjkey in 'p061898' or subjkey in 'p069857':
      continue
    if subjkey in folder_list:
        for timekey in all_data_dic[subjkey].keys():
            dl_dir = 'xxx/MIMICIII/'
            data_path  = dl_dir + subjkey[:3]+'/'+subjkey+'/'+subjkey+'-'+timekey
            record = wfdb.rdrecord(data_path) 
            ppg_sig = []
            for i in record.p_signal:
                ppg_sig.append(i[record.sig_name.index('PLETH')])
                
            for time_data in all_data_dic[subjkey][timekey]:
                time_data=time_data.replace('--','-')
                split_data = time_data.split('-')
                st = split_data[0]
                xt = split_data[1]
                var = ("hours","minutes","seconds")
                time2sec = lambda x:int(datetime.timedelta(**{k:int(v) for k,v in zip(var,x.strip().split(":"))}).total_seconds())   
                
                app_sig = ppg_sig[time2sec(st)*125:time2sec(xt)*125]
                app_sig = np.array(app_sig)
                app_sig = app_sig[~np.isnan(app_sig)]
                data_dic[subjkey+"-"+timekey+"-"+time_data]=app_sig
            

    import pickle
    file_open=open('xxx/mimicdata.pl','wb')
    pickle.dump((all_data_dic,data_dic),file_open)
    file_open.close()


In [ ]:
key_list = list(data_dic.keys())
same_data = []
last_data = ''
last_time = ''
isfirst = True
for i in key_list:
  split_data = i.split('-')
  subj_time = '-'.join(split_data[:6])
  
  if subj_time == last_data:
    if isfirst:
      same_data.append((last_data, last_time))
      same_data.append((subj_time,'-'.join(split_data[6:] )))
      isfirst=False
    else:
      same_data.append((subj_time,'-'.join(split_data[6:] )))
  else:
    isfirst = True
    
  last_data = subj_time
  last_time = '-'.join(split_data[6:])
  
same_data.sort()

all_data_dic  = {}
key_list = list(data_dic.keys())
for i in key_list:
  split_data = i.split('-')
  subj_time = '-'.join(split_data[:6])
   
  if subj_time in all_data_dic.keys():
    index = len(all_data_dic[subj_time])
    all_data_dic[subj_time][index]=[] 
    all_data_dic[subj_time][index].append('-'.join(split_data[6:]))
  else:
    all_data_dic[subj_time]={}
    all_data_dic[subj_time][0]=[]
    all_data_dic[subj_time][0].append('-'.join(split_data[6:]))


same_dic = {}
for i in same_data:
  if i[0] in same_dic.keys():
    same_dic[i[0]].append(i[1])
  else:
    same_dic[i[0]]=[]
    same_dic[i[0]].append(i[1])

import datetime

my_dic = {}

for i in same_dic.keys():
  my_dic[i]={}
  number = 0
  my_dic[i][number]=[]
  last_time = 0
  for j in same_dic[i]:
    
    st = j.split('-')[0]
    xt = j.split('-')[1]
    var = ("hours","minutes","seconds")
    time2sec = lambda x:int(datetime.timedelta(**{k:int(v) for k,v in zip(var,x.strip().split(":"))}).total_seconds())
    sub_time = (time2sec(xt)-time2sec(st))+ last_time

    if sub_time>=180:
      my_dic[i][number].append(j)
      last_time = 0
      number+=1
      my_dic[i][number]=[]
    else:
      my_dic[i][number].append(j)
      last_time = sub_time


# new_dic ={}
dic_list = list(my_dic.keys())

for i in dic_list:
  list_i =  list(my_dic[i].keys())
  for j in list_i:
    if my_dic[i][j]==[]:
      del my_dic[i][j]

for i in dic_list:
  list_i =  list(my_dic[i].keys())
  if len(list_i)==1:
    continue
  last_dic = my_dic[i][0]
  for j in list_i[:-1]:
    st = last_dic[-1].split('-')[1]
    xt = my_dic[i][j+1][0].split('-')[0]
    var = ("hours","minutes","seconds")
    time2sec = lambda x:int(datetime.timedelta(**{k:int(v) for k,v in zip(var,x.strip().split(":"))}).total_seconds())
    sub_time = (time2sec(xt)-time2sec(st)) + last_time
    if sub_time >= 36000 or sub_time <= -36000:
      last_dic=my_dic[i][j+1]
      continue
    else:
      my_dic[i][j] = my_dic[i][j]+my_dic[i][j+1]
      my_dic[i][j+1]=[]
      last_dic = my_dic[i][j]


dic_list = list(my_dic.keys())
for i in dic_list:
  list_i =  list(my_dic[i].keys())
  for j in list_i:
    if my_dic[i][j]==[]:
      del my_dic[i][j]

for i in my_dic:
    all_data_dic[i]=my_dic[i]

sig_dic = {}
for i in all_data_dic.keys():
  sig_dic[i]={}
  for j in all_data_dic[i].keys():
    sig_arr = []
    for k in all_data_dic[i][j]:
      sig_arr += data_dic[i+'-'+k].tolist()
    sig_dic[i][j]=sig_arr

import pickle
file_open=open('xxxx/mimicdata_final.pl','wb')
pickle.dump((all_data_dic,sig_dic),file_open)
file_open.close()

# Split single cycle data


In [ ]:
import numpy as np
from scipy.signal import find_peaks, stft, lfilter, butter, welch
from plotly.subplots import make_subplots
from plotly.colors import n_colors
import plotly.graph_objects as go
from scipy.interpolate import interp1d


In [ ]:

import numpy as np
from scipy.signal import find_peaks, stft, lfilter, butter, welch
from plotly.subplots import make_subplots
from plotly.colors import n_colors
import plotly.graph_objects as go
from scipy.interpolate import interp1d


class BVPsignal:
    """
        Manage (multi-channel, row-wise) BVP signals
    """
    nFFT = 2048  # freq. resolution for STFTs
    step = 1       # step in seconds

    def __init__(self, data, fs, startTime=0, minHz=0.75, maxHz=4., verb=False):
        if len(data.shape) == 1:
            self.data = data.reshape(1,-1) # 2D array raw-wise
        else:
            self.data = data
        self.numChls = self.data.shape[0]  # num  channels
        self.fs = fs                       # sample rate
        self.startTime = startTime
        self.verb = verb
        self.minHz = minHz
        self.maxHz = maxHz

    def getChunk(startTime, winsize=None, numSample=None):
        
        assert startTime >= self.startTime, "Start time error!"
        
        N = self.data.shape[1] 
        fs = self.fs
        Nstart = int(fs*startTime)
        
        # -- winsize > 0
        if winsize:
            stopTime = startTime + winsize
            Nstop = np.min([int(fs*stopTime),N])
            
        # -- numSample > 0
        if numSample:
            Nstop = np.min([numSample,N])
        
        return self.data[0,Nstart:Nstop]
        
    def hps(self, spect, d=3):
        
        if spect.ndim == 2:
            n_win = spect.shape[1]
            new_spect = np.zeros_like(spect)
            for w in range(n_win):
                curr_w = spect[:,w]
                w_down_z = np.zeros_like(curr_w)
                w_down = curr_w[::d]
                w_down_z[0:len(w_down)] = w_down
                w_hps = np.multiply(curr_w, w_down_z)
                new_spect[:, w] = w_hps
            return new_spect

        elif spect.ndim == 1:
            s_down_z = np.zeros_like(spect)
            s_down = spect[::d]
            s_down_z[0:len(s_down)] = s_down
            w_hps = np.multiply(spect, s_down_z)
            return w_hps

        else:
            raise ValueError("Wrong Dimensionality of the Spectrogram for the HPS")

    def spectrogram(self, winsize=5, use_hps=False):
        """
        Compute the BVP signal spectrogram restricted to the
        band 42-240 BPM by using winsize (in sec) samples.
        """

        # -- spect. Z is 3-dim: Z[#chnls, #freqs, #times]
        F, T, Z = stft(self.data,
                       self.fs,
                       nperseg=self.fs*winsize,
                       noverlap=self.fs*(winsize-self.step),
                       boundary='even',
                       nfft=self.nFFT)
        Z = np.squeeze(Z, axis=0)

        # -- freq subband (0.75 Hz - 4.0 Hz)
        minHz = 0.75
        maxHz = 4.0
        band = np.argwhere((F > minHz) & (F < maxHz)).flatten()
        self.spect = np.abs(Z[band,:])     # spectrum magnitude
        self.freqs = 60*F[band]            # spectrum freq in bpm
        self.times = T                     # spectrum times

        if use_hps:
            spect_hps = self.hps(self.spect)
            # -- BPM estimate by spectrum
            self.bpm = self.freqs[np.argmax(spect_hps,axis=0)]
        else:
            # -- BPM estimate by spectrum
            self.bpm = self.freqs[np.argmax(self.spect,axis=0)]
        
    def getBPM(self, winsize=5):
        self.spectrogram(winsize, use_hps=False)
        return self.bpm, self.times
    
    def PSD2BPM(self, chooseBest=True, use_hps=False):
        """
            Compute power spectral density using Welch’s method and estimate
            BPMs from video frames
        """

        # -- interpolation for less than 256 samples
        c,n = self.data.shape
        if n < 256:
            seglength = n
            overlap = int(0.8*n)  # fixed overlapping
        else:
            seglength = 256
            overlap = 200
       
        # -- periodogram by Welch
        F, P = welch(self.data, nperseg=seglength, noverlap=overlap, window='hamming',fs=self.fs, nfft=self.nFFT)

        # -- freq subband (0.75 Hz - 4.0 Hz)
        band = np.argwhere((F > self.minHz) & (F < self.maxHz)).flatten()
        self.Pfreqs = 60*F[band]
        self.Power = P[:,band]
        
        # -- if c = 3 choose that with the best SNR
        if chooseBest:
            winner = 0
            lobes = self.PDSrippleAnalysis(ch=0)
            SNR = lobes[-1]/lobes[-2]
            if c == 3:
                lobes = self.PDSrippleAnalysis(ch=1)
                SNR1 = lobes[-1]/lobes[-2]
                if SNR1 > SNR:
                    SNR = SNR1
                    winner = 1
                lobes = self.PDSrippleAnalysis(ch=2)
                SNR1 = lobes[-1]/lobes[-2]
                if SNR1 > SNR:
                    SNR = SNR1
                    winner = 2    
            self.Power = self.Power[winner].reshape(1,-1)
        
        # TODO: eliminare?
        if use_hps:
            p = self.Power[0]
            phps = self.hps(p)
            '''import matplotlib.pyplot as plt
            plt.plot(p)
            plt.figure()
            plt.plot(phps)
            plt.show()'''
            Pmax = np.argmax(phps)  # power max
            self.bpm = np.array([self.Pfreqs[Pmax]])       # freq max

        else:
            # -- BPM estimate by PSD
            Pmax = np.argmax(self.Power, axis=1)  # power max
            self.bpm = self.Pfreqs[Pmax]       # freq max

        if '3' in str(self.verb):
            lobes = self.PDSrippleAnalysis()
            self.displayPSD(lobe1=lobes[-1], lobe2=lobes[-2])

    def autocorr(self):
        from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

        # TODO: to handle all channels
        x = self.data[0,:]
        plot_acf(x)
        plt.show()

        plot_pacf(x)
        plt.show()

    def displaySpectrum(self, display=False, dims=3):
        """Show the spectrogram of the BVP signal"""

        # -- check if bpm exists
        try:
            bpm = self.bpm
        except AttributeError:
            self.spectrogram()
            bpm = self.bpm
            
        t = self.times
        f = self.freqs
        S = self.spect

        fig = go.Figure()
        fig.add_trace(go.Heatmap(z=S, x=t, y=f, colorscale="viridis"))
        fig.add_trace(go.Scatter(x=t, y=bpm, name='Frequency Domain', line=dict(color='red', width=2)))

        fig.update_layout(autosize=False, height=420, showlegend=True,
                          title='Spectrogram of the BVP signal',
                          xaxis_title='Time (sec)',
                          yaxis_title='BPM (60*Hz)',
                          legend=dict(
                            x=0,
                            y=1,
                            traceorder="normal",
                            font=dict(
                                family="sans-serif",
                                size=12,
                                color="black"),
                            bgcolor="LightSteelBlue",
                            bordercolor="Black",
                            borderwidth=2)
                         )
                       
        fig.show()

    def findPeaks(self, distance=None, height=None):
        
        # -- take the first channel
        x = self.data[0].flatten()
            
        if distance is None:
            distance = self.fs/2
        if height is None:
            height = np.mean(x)

        # -- find peaks with the specified params
        self.peaks, _ = find_peaks(x, distance=distance, height=height)
        
        self.peaksTimes = self.peaks/self.fs
        self.bpmPEAKS = 60.0/np.diff(self.peaksTimes)
        
    def plotBPMPeaks(self, height=None, width=None):
        """
            Plot the the BVP signal and peak marks
        """

        # -- find peaks  
        try:
            peaks = self.peaks
        except AttributeError:
            self.findPeaks()
            peaks = self.peaks
        
        #-- signals 
        y = self.data[0]
        n = y.shape[0]
        startTime  = self.startTime 
        stopTime = startTime+n/self.fs
        x = np.linspace(startTime, stopTime, num=n, endpoint=False)
        
        fig = go.Figure()
        fig.add_trace(go.Scatter(x=x, y=y, name="BVP"))
        fig.add_trace(go.Scatter(x=x[peaks], y=y[peaks], mode='markers', name="Peaks"))

        if not height:
            height=400
        if not width:
            width=800

        fig.update_layout(height=height, width=width, title="BVP signal + peaks",
            font=dict(
                family="Courier New, monospace",
                size=14,
                color="#7f7f7f"))
        
        fig.show()
        
    def plot(self, title="BVP signal", height=400, width=800):
        """
            Plot the the BVP signal (multiple channels)
        """
      
        #-- signals 
        y = self.data
        c,n = y.shape
        startTime  = self.startTime 
        stopTime = startTime+n/self.fs
        x = np.linspace(startTime, stopTime, num=n, endpoint=False)
        
        fig = go.Figure()
        
        for i in range(c):
            name = "BVP " + str(i)
            fig.add_trace(go.Scatter(x=x, y=y[i], name=name))

        fig.update_layout(height=height, width=width, title=title,
            font=dict(
                family="Courier New, monospace",
                size=14,
                color="#7f7f7f"))
        fig.show()
        
    def displayPSD(self, ch=0, lobe1=None, lobe2=None, GT=None):
        """Show the periodogram(s) of the BVP signal for channel ch"""

        f = self.Pfreqs 
        P = self.Power[ch] 
                
        fig = go.Figure()
        
        fig.add_trace(go.Scatter(x=f, y=P, name='PSD'))
        fig.update_layout(autosize=False, width=500, height=400)
        
        if lobe1 is not None and lobe2 is not None:
            L1 = lobe1
            L2 = lobe2
            # Add horiz. lobe peack lines
            fig.add_shape(type="line",x0=f[0], y0=L1, x1=f[-1], y1=L1,
                line=dict(color="LightSeaGreen", width=2, dash="dashdot"))
            fig.add_shape(type="line",x0=f[0], y0=L2, x1=f[-1], y1=L2,
                line=dict(color="SeaGreen", width=2, dash="dashdot"))
            tit = 'SNR = ' + str(np.round(L1/L2,2))
            fig.update_layout(title=tit)
            
        if GT is not None:
            # Add vertical GT line
            fig.add_shape(type="line",x0=GT, y0=0, x1=GT, y1=np.max(P),
                line=dict(color="DarkGray", width=2, dash="dash"))
            
        fig.show()
  
    def PDSrippleAnalysis(self, ch=0):
        # -- ripple analysis
        
        P = self.Power[ch].flatten()
        dP = np.gradient(P)
        n = len(dP)
        I = []; 
        i = 0
        while i < n:
            m = 0
            # -- positive gradient
            while (i < n) and (dP[i] > 0):
                m = max([m,P[i]])
                i += 1
            I.append(m)
            # -- skip negative gradient
            while (i < n) and (dP[i] < 0) : 
                i += 1
        lobes = np.sort(I)
        if len(lobes) < 2:
            lobes = np.array([lobes,0])
            
        return lobes

In [ ]:
import numpy as np
import ast
import plotly.graph_objects as go
import scipy
from scipy import stats
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
mm = MinMaxScaler()
from scipy.signal import medfilt, detrend
from abc import ABCMeta, abstractmethod
from importlib import import_module


In [ ]:
from tqdm import tqdm

split_data_dic = {}
for subj in tqdm(sig_dic.keys()):
  split_data_dic[subj]={}
  for timed in sig_dic[subj].keys():
    
    cycle_ppg=[]
    bvpChunk = BVPsignal(np.array(sig_dic[subj][timed]), 125, 0)
    bvpChunk.findPeaks()
    low_point = []
    before = 0
    minnumber = 0 
    for i in bvpChunk.peaks:
      index_ = np.argmin(bvpChunk.data[0][before:i])
      low_point.append(index_+before)
      before = i

    index_ = np.argmin(bvpChunk.data[0][before:-1])
    low_point.append(index_+before)

    
    for index, i in  enumerate(low_point):
      if index==0:
        continue 
      cycle_ppg.append(bvpChunk.data[0][low_point[index-1]:low_point[index]])


    split_data_dic[subj][timed]=cycle_ppg


import pickle
file_open=open('/content/drive/MyDrive/PhD/Letter/mimicdata_final_v.pl','wb')
pickle.dump((subj_dic,sig_dic,split_data_dic),file_open)
file_open.close()
